In [5]:
import pandas as pd

In [7]:
test_1= pd.read_csv('heart_test.csv')

In [8]:
test_1.head()

,Number,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,179,57,1,0,150,276,0,0,112,1,0.6,1,1,1,0
1,228,59,1,3,170,288,0,0,159,0,0.2,1,0,3,0
2,111,57,1,2,150,126,1,1,173,0,0.2,2,1,3,1
3,246,56,0,0,134,409,0,0,150,1,1.9,1,2,3,0
4,60,71,0,2,110,265,1,0,130,0,0.0,2,1,2,1


In [9]:
test_1.to_csv('test_number.csv', index = False, header = False)